In [ ]:
# Data Wrangling
import pandas as pd
import numpy as np
import pandas_profiling as pp
import gc

# Visualization, EDA
import missingno as msno
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# Korean font
#font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
#rc('font', family=font_name)

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import optuna

# Modeling
from lightgbm import LGBMClassifier
from pycaret.classification import *
from sklearn.dummy import DummyClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import *
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import ClassifierMixin

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from keras.callbacks import ModelCheckpoint

# Utility
import re
import os
import time
import datetime
import random
import tensorflow as tf
import pickle
import joblib
import platform
import warnings; warnings.filterwarnings("ignore")
from itertools import combinations
from tqdm import tqdm, tqdm_notebook
from IPython.display import Image
from scipy.stats.mstats import gmean
from tensorflow import keras
from PIL import Image

from sklearn.datasets import make_classification

from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold

# 1. test_id  불러오기

In [ ]:
# 주피터 파일일때(주석처리)
panel = pd.read_csv(os.path.abspath("../input") + '/panel.csv')
survey = pd.read_csv(os.path.abspath("../input") + '/survey.csv')
response_train = pd.read_csv(os.path.abspath("../input") + '/response_train.csv')
response_test = pd.read_csv(os.path.abspath("../input") + '/response_test.csv')

all_train = response_train.merge(panel).merge(survey)
test = response_test.merge(panel).merge(survey).sort_values(by='ID')

test_id = test.iloc[:,0:1]

In [ ]:
del panel, survey, response_train, response_test
gc.collect()

20

# 2-1. 모델링(피쳐 변형 전) -다소 시간이 많이많이 소요됨

## (1월) 모델링

In [ ]:
# 쓰는 데이터셋만 빼고 주석 처리

X_train = joblib.load(os.path.abspath("../input") + "/X_train(01월).pkl")
y_train = joblib.load(os.path.abspath("../input") + "/y_train(01월).pkl")
X_test = joblib.load(os.path.abspath("../input") + "/X_test(01월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train(02월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train(02월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test(02월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train(03월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train(03월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test(03월).pkl")

### BaggingClassifier(CatBoostClassifier)

In [ ]:
########BaggingClassifier(CatBoostClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

cat = BaggingClassifier(base_estimator = CatBoostClassifier(), random_state=42)

cat_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    cat.fit(X_tr, y_tr)
    
    pred = cat.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(cat.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    cat_pred += sub_pred

In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  cat_pred}).to_csv(os.path.abspath("../submission")+ '/CAT(변경전_1월)_0.9294.csv', index=False)

### BaggingClassifier(RandomForestClassifier)

In [ ]:
########BaggingClassifier(RandomForestClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

rf = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state=42)

rf_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    rf.fit(X_tr, y_tr)
    
    pred = rf.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(rf.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    rf_pred += sub_pred

BaggingClassifier 평균 AUC : 0.9314623473590823


In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  rf_pred}).to_csv(os.path.abspath("../submission")+ '/RF(변경전_1월)_0.9314.csv', index=False)

### Early_stopping(LGBMClassifier)

In [ ]:
########Early_stopping(LGBMClassifier)2등 코드########

X_tr, X_dev, y_tr, y_dev = train_test_split(X_train, y_train, test_size=0.4, random_state=77) # test size 올리기위해 0.4

evals = [(X_dev, y_dev)] 

lgbm_wrapper = LGBMClassifier(n_estimators = 400,random_state=42) # 400번 학습

lgbm_wrapper.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_dev)
lgbm_pred_proba = lgbm_wrapper.predict_proba(X_dev)[:,1]

[1]	valid_0's binary_logloss: 0.641336
[2]	valid_0's binary_logloss: 0.604014
[3]	valid_0's binary_logloss: 0.573049
[4]	valid_0's binary_logloss: 0.547087
[5]	valid_0's binary_logloss: 0.525168
[6]	valid_0's binary_logloss: 0.506481
[7]	valid_0's binary_logloss: 0.490527
[8]	valid_0's binary_logloss: 0.476838
[9]	valid_0's binary_logloss: 0.465075
[10]	valid_0's binary_logloss: 0.454934
[11]	valid_0's binary_logloss: 0.446093
[12]	valid_0's binary_logloss: 0.438436
[13]	valid_0's binary_logloss: 0.431778
[14]	valid_0's binary_logloss: 0.425994
[15]	valid_0's binary_logloss: 0.42094
[16]	valid_0's binary_logloss: 0.416496
[17]	valid_0's binary_logloss: 0.412608
[18]	valid_0's binary_logloss: 0.409175
[19]	valid_0's binary_logloss: 0.406215
[20]	valid_0's binary_logloss: 0.403563
[21]	valid_0's binary_logloss: 0.401233
[22]	valid_0's binary_logloss: 0.399073
[23]	valid_0's binary_logloss: 0.397271
[24]	valid_0's binary_logloss: 0.39513
[25]	valid_0's binary_logloss: 0.393572
[26]	valid_

[206]	valid_0's binary_logloss: 0.353199
[207]	valid_0's binary_logloss: 0.353158
[208]	valid_0's binary_logloss: 0.353163
[209]	valid_0's binary_logloss: 0.353119
[210]	valid_0's binary_logloss: 0.353114
[211]	valid_0's binary_logloss: 0.353052
[212]	valid_0's binary_logloss: 0.352977
[213]	valid_0's binary_logloss: 0.352927
[214]	valid_0's binary_logloss: 0.35284
[215]	valid_0's binary_logloss: 0.352736
[216]	valid_0's binary_logloss: 0.352675
[217]	valid_0's binary_logloss: 0.352663
[218]	valid_0's binary_logloss: 0.352609
[219]	valid_0's binary_logloss: 0.352525
[220]	valid_0's binary_logloss: 0.352494
[221]	valid_0's binary_logloss: 0.35234
[222]	valid_0's binary_logloss: 0.352233
[223]	valid_0's binary_logloss: 0.352203
[224]	valid_0's binary_logloss: 0.352201
[225]	valid_0's binary_logloss: 0.352145
[226]	valid_0's binary_logloss: 0.352133
[227]	valid_0's binary_logloss: 0.352035
[228]	valid_0's binary_logloss: 0.35201
[229]	valid_0's binary_logloss: 0.351947
[230]	valid_0's bin

In [ ]:
from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred) 
    f1 = f1_score(y_test, pred) 
    # ROC AUC 
    roc_auc = roc_auc_score(y_test, pred_proba) 
    
    print('Confusion Matrix') 
    print(confusion) 
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format( accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_dev, lgbm_preds, lgbm_pred_proba)

Confusion Matrix
[[322947  53894]
 [ 52456 250179]]
accuracy: 0.8435, precision: 0.8228, recall: 0.8267, f1: 0.8247, roc_auc: 0.9246


In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  lgbm_wrapper.predict_proba(X_test)[:,1]}).to_csv(os.path.abspath("../submission")+ '/LGBM(변경전_1월)_0.9246.csv', index=False)

In [ ]:
joblib.dump(lgbm_wrapper, os.path.abspath("../models") + "/LGBM(변경전_1월).pkl")

['C:\\Users\\dmsdn\\학교수업\\머신러닝\\컴피티션\\ML 컴피티션_김은욱서하은\\models/LGBM(변경전_1월).pkl']

## (2월) 모델링

In [ ]:
# 쓰는 데이터셋만 빼고 주석 처리

#X_train = joblib.load(os.path.abspath("../input") + "/X_train(01월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train(01월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test(01월).pkl")

X_train = joblib.load(os.path.abspath("../input") + "/X_train(02월).pkl")
y_train = joblib.load(os.path.abspath("../input") + "/y_train(02월).pkl")
X_test = joblib.load(os.path.abspath("../input") + "/X_test(02월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train(03월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train(03월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test(03월).pkl")

### BaggingClassifier(CatBoostClassifier)

In [ ]:
########BaggingClassifier(CatBoostClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

cat = BaggingClassifier(base_estimator = CatBoostClassifier(), random_state=42)

cat_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    cat.fit(X_tr, y_tr)
    
    pred = cat.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(cat.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    cat_pred += sub_pred

In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  cat_pred}).to_csv(os.path.abspath("../submission")+ '/CAT(변경전_2월)_0.9285.csv', index=False)

### BaggingClassifier(RandomForestClassifier)

In [ ]:
########BaggingClassifier(RandomForestClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

rf = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state=42)

rf_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    rf.fit(X_tr, y_tr)
    
    pred = rf.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(rf.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    rf_pred += sub_pred

BaggingClassifier 평균 AUC : 0.9314623473590823


In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  rf_pred}).to_csv(os.path.abspath("../submission")+ '/RF(변경전_2월)_0.9243.csv', index=False)

### Early_stopping(LGBMClassifier)

In [ ]:
########Early_stopping(LGBMClassifier)2등 코드########

X_tr, X_dev, y_tr, y_dev = train_test_split(X_train, y_train, test_size=0.4, random_state=77) # test size 올리기위해 0.4

evals = [(X_dev, y_dev)] 

lgbm_wrapper = LGBMClassifier(n_estimators = 400,random_state=42) # 400번 학습

lgbm_wrapper.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_dev)
lgbm_pred_proba = lgbm_wrapper.predict_proba(X_dev)[:,1]

[1]	valid_0's binary_logloss: 0.641072
[2]	valid_0's binary_logloss: 0.604177
[3]	valid_0's binary_logloss: 0.573541
[4]	valid_0's binary_logloss: 0.547893
[5]	valid_0's binary_logloss: 0.526178
[6]	valid_0's binary_logloss: 0.507766
[7]	valid_0's binary_logloss: 0.491989
[8]	valid_0's binary_logloss: 0.478478
[9]	valid_0's binary_logloss: 0.466817
[10]	valid_0's binary_logloss: 0.456671
[11]	valid_0's binary_logloss: 0.447908
[12]	valid_0's binary_logloss: 0.440332
[13]	valid_0's binary_logloss: 0.433718
[14]	valid_0's binary_logloss: 0.427692
[15]	valid_0's binary_logloss: 0.422651
[16]	valid_0's binary_logloss: 0.418069
[17]	valid_0's binary_logloss: 0.414028
[18]	valid_0's binary_logloss: 0.410338
[19]	valid_0's binary_logloss: 0.40702
[20]	valid_0's binary_logloss: 0.404337
[21]	valid_0's binary_logloss: 0.401651
[22]	valid_0's binary_logloss: 0.399114
[23]	valid_0's binary_logloss: 0.396895
[24]	valid_0's binary_logloss: 0.395123
[25]	valid_0's binary_logloss: 0.393161
[26]	valid

[206]	valid_0's binary_logloss: 0.348844
[207]	valid_0's binary_logloss: 0.348808
[208]	valid_0's binary_logloss: 0.34872
[209]	valid_0's binary_logloss: 0.348686
[210]	valid_0's binary_logloss: 0.348584
[211]	valid_0's binary_logloss: 0.348496
[212]	valid_0's binary_logloss: 0.348446
[213]	valid_0's binary_logloss: 0.348374
[214]	valid_0's binary_logloss: 0.348291
[215]	valid_0's binary_logloss: 0.348242
[216]	valid_0's binary_logloss: 0.348218
[217]	valid_0's binary_logloss: 0.348165
[218]	valid_0's binary_logloss: 0.348097
[219]	valid_0's binary_logloss: 0.348091
[220]	valid_0's binary_logloss: 0.348073
[221]	valid_0's binary_logloss: 0.348058
[222]	valid_0's binary_logloss: 0.348015
[223]	valid_0's binary_logloss: 0.347977
[224]	valid_0's binary_logloss: 0.347911
[225]	valid_0's binary_logloss: 0.347805
[226]	valid_0's binary_logloss: 0.347793
[227]	valid_0's binary_logloss: 0.347782
[228]	valid_0's binary_logloss: 0.347747
[229]	valid_0's binary_logloss: 0.347691
[230]	valid_0's b

In [ ]:
from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred) 
    f1 = f1_score(y_test, pred) 
    # ROC AUC 
    roc_auc = roc_auc_score(y_test, pred_proba) 
    
    print('Confusion Matrix') 
    print(confusion) 
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format( accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_dev, lgbm_preds, lgbm_pred_proba)

Confusion Matrix
[[233387  38530]
 [ 37008 178079]]
accuracy: 0.8449, precision: 0.8221, recall: 0.8279, f1: 0.8250, roc_auc: 0.9259


In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  lgbm_wrapper.predict_proba(X_test)[:,1]}).to_csv(os.path.abspath("../submission")+ '/LGBM(변경전_2월)_0.9259.csv', index=False)

In [ ]:
joblib.dump(lgbm_wrapper, os.path.abspath("../models") + "/LGBM(변경전_2월).pkl")

['C:\\Users\\dmsdn\\학교수업\\머신러닝\\컴피티션\\ML 컴피티션_김은욱서하은\\models/LGBM(변경전_2월).pkl']

## (3월) 모델링

In [ ]:
# 쓰는 데이터셋만 빼고 주석 처리

#X_train = joblib.load(os.path.abspath("../input") + "/X_train(01월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train(01월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test(01월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train(02월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train(02월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test(02월).pkl")

X_train = joblib.load(os.path.abspath("../input") + "/X_train(03월).pkl")
y_train = joblib.load(os.path.abspath("../input") + "/y_train(03월).pkl")
X_test = joblib.load(os.path.abspath("../input") + "/X_test(03월).pkl")

### BaggingClassifier(CatBoostClassifier)

In [ ]:
########BaggingClassifier(CatBoostClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

cat = BaggingClassifier(base_estimator = CatBoostClassifier(), random_state=42)

cat_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    cat.fit(X_tr, y_tr)
    
    pred = cat.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(cat.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    cat_pred += sub_pred

In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  cat_pred}).to_csv(os.path.abspath("../submission")+ '/CAT(변경전_3월)_0.9308.csv', index=False)

### BaggingClassifier(RandomForestClassifier)

In [ ]:
########BaggingClassifier(RandomForestClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

rf = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state=42)

rf_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    rf.fit(X_tr, y_tr)
    
    pred = rf.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(rf.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    rf_pred += sub_pred

BaggingClassifier 평균 AUC : 0.9314623473590823


In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  rf_pred}).to_csv(os.path.abspath("../submission")+ '/RF(변경전_3월)_0.9264.csv', index=False)

## Early_stopping(LGBMClassifier)

In [ ]:
########Early_stopping(LGBMClassifier)2등 코드########

X_tr, X_dev, y_tr, y_dev = train_test_split(X_train, y_train, test_size=0.4, random_state=77) # test size 올리기위해 0.4

evals = [(X_dev, y_dev)] 

lgbm_wrapper = LGBMClassifier(n_estimators = 400,random_state=42) # 400번 학습

lgbm_wrapper.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_dev)
lgbm_pred_proba = lgbm_wrapper.predict_proba(X_dev)[:,1]

[1]	valid_0's binary_logloss: 0.643344
[2]	valid_0's binary_logloss: 0.605932
[3]	valid_0's binary_logloss: 0.574847
[4]	valid_0's binary_logloss: 0.548762
[5]	valid_0's binary_logloss: 0.526319
[6]	valid_0's binary_logloss: 0.507586
[7]	valid_0's binary_logloss: 0.491563
[8]	valid_0's binary_logloss: 0.477689
[9]	valid_0's binary_logloss: 0.465805
[10]	valid_0's binary_logloss: 0.455443
[11]	valid_0's binary_logloss: 0.446317
[12]	valid_0's binary_logloss: 0.438309
[13]	valid_0's binary_logloss: 0.431282
[14]	valid_0's binary_logloss: 0.425285
[15]	valid_0's binary_logloss: 0.419778
[16]	valid_0's binary_logloss: 0.414893
[17]	valid_0's binary_logloss: 0.410544
[18]	valid_0's binary_logloss: 0.406677
[19]	valid_0's binary_logloss: 0.403305
[20]	valid_0's binary_logloss: 0.40044
[21]	valid_0's binary_logloss: 0.396851
[22]	valid_0's binary_logloss: 0.394433
[23]	valid_0's binary_logloss: 0.39204
[24]	valid_0's binary_logloss: 0.389989
[25]	valid_0's binary_logloss: 0.387973
[26]	valid_

[203]	valid_0's binary_logloss: 0.342119
[204]	valid_0's binary_logloss: 0.342087
[205]	valid_0's binary_logloss: 0.342052
[206]	valid_0's binary_logloss: 0.342037
[207]	valid_0's binary_logloss: 0.341967
[208]	valid_0's binary_logloss: 0.341904
[209]	valid_0's binary_logloss: 0.341856
[210]	valid_0's binary_logloss: 0.341821
[211]	valid_0's binary_logloss: 0.341827
[212]	valid_0's binary_logloss: 0.341806
[213]	valid_0's binary_logloss: 0.341742
[214]	valid_0's binary_logloss: 0.341716
[215]	valid_0's binary_logloss: 0.341695
[216]	valid_0's binary_logloss: 0.34165
[217]	valid_0's binary_logloss: 0.341644
[218]	valid_0's binary_logloss: 0.341585
[219]	valid_0's binary_logloss: 0.341519
[220]	valid_0's binary_logloss: 0.341444
[221]	valid_0's binary_logloss: 0.341423
[222]	valid_0's binary_logloss: 0.341398
[223]	valid_0's binary_logloss: 0.341379
[224]	valid_0's binary_logloss: 0.341349
[225]	valid_0's binary_logloss: 0.3413
[226]	valid_0's binary_logloss: 0.34125
[227]	valid_0's bina

In [ ]:
from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred) 
    f1 = f1_score(y_test, pred) 
    # ROC AUC 
    roc_auc = roc_auc_score(y_test, pred_proba) 
    
    print('Confusion Matrix') 
    print(confusion) 
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format( accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_dev, lgbm_preds, lgbm_pred_proba)

Confusion Matrix
[[171141  28994]
 [ 27136 140114]]
accuracy: 0.8472, precision: 0.8285, recall: 0.8378, f1: 0.8331, roc_auc: 0.9286


In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  lgbm_wrapper.predict_proba(X_test)[:,1]}).to_csv(os.path.abspath("../submission")+ '/LGBM(변경전_3월)_0.9286.csv', index=False)

In [ ]:
joblib.dump(lgbm_wrapper, os.path.abspath("../models") + "/LGBM(변경전_3월).pkl")

['C:\\Users\\dmsdn\\학교수업\\머신러닝\\컴피티션\\ML 컴피티션_김은욱서하은\\models/LGBM(변경전_3월).pkl']

# 2-2. 모델링(피쳐 변형 후) -다소 시간이 많이많이 소요됨

## (1월) 모델링

In [ ]:
# 쓰는 데이터셋만 빼고 주석 처리

X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(01월).pkl")
y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(01월).pkl")
X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(01월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(02월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(02월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(02월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(03월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(03월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(03월).pkl")

### BaggingClassifier(CatBoostClassifier)

In [ ]:
########BaggingClassifier(CatBoostClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

cat = BaggingClassifier(base_estimator = CatBoostClassifier(), random_state=42)

cat_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    cat.fit(X_tr, y_tr)
    
    pred = cat.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(cat.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    cat_pred += sub_pred

In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  cat_pred}).to_csv(os.path.abspath("../submission")+ '/CAT(피쳐변경_1월)_0.9535.csv', index=False)

### BaggingClassifier(RandomForestClassifier)

In [ ]:
########BaggingClassifier(RandomForestClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

rf = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state=42)

rf_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    rf.fit(X_tr, y_tr)
    
    pred = rf.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(rf.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    rf_pred += sub_pred

BaggingClassifier 평균 AUC : 0.9314623473590823


In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  rf_pred}).to_csv(os.path.abspath("../submission")+ '/RF(피쳐변경_1월)_0.9449.csv', index=False)

### Early_stopping(LGBMClassifier)

In [ ]:
########Early_stopping(LGBMClassifier)2등 코드########

X_tr, X_dev, y_tr, y_dev = train_test_split(X_train, y_train, test_size=0.4, random_state=77) # test size 올리기위해 0.4

evals = [(X_dev, y_dev)] 

lgbm_wrapper = LGBMClassifier(n_estimators = 400,random_state=42) # 400번 학습

lgbm_wrapper.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_dev)
lgbm_pred_proba = lgbm_wrapper.predict_proba(X_dev)[:,1]

[1]	valid_0's binary_logloss: 0.61627
[2]	valid_0's binary_logloss: 0.576782
[3]	valid_0's binary_logloss: 0.54425
[4]	valid_0's binary_logloss: 0.51688
[5]	valid_0's binary_logloss: 0.493733
[6]	valid_0's binary_logloss: 0.474065
[7]	valid_0's binary_logloss: 0.457149
[8]	valid_0's binary_logloss: 0.442467
[9]	valid_0's binary_logloss: 0.430967
[10]	valid_0's binary_logloss: 0.419875
[11]	valid_0's binary_logloss: 0.410797
[12]	valid_0's binary_logloss: 0.402405
[13]	valid_0's binary_logloss: 0.394662
[14]	valid_0's binary_logloss: 0.388851
[15]	valid_0's binary_logloss: 0.383163
[16]	valid_0's binary_logloss: 0.378021
[17]	valid_0's binary_logloss: 0.373211
[18]	valid_0's binary_logloss: 0.369164
[19]	valid_0's binary_logloss: 0.365732
[20]	valid_0's binary_logloss: 0.362286
[21]	valid_0's binary_logloss: 0.359519
[22]	valid_0's binary_logloss: 0.356379
[23]	valid_0's binary_logloss: 0.353905
[24]	valid_0's binary_logloss: 0.351617
[25]	valid_0's binary_logloss: 0.349565
[26]	valid_0

[203]	valid_0's binary_logloss: 0.296995
[204]	valid_0's binary_logloss: 0.296934
[205]	valid_0's binary_logloss: 0.296876
[206]	valid_0's binary_logloss: 0.29681
[207]	valid_0's binary_logloss: 0.296759
[208]	valid_0's binary_logloss: 0.296612
[209]	valid_0's binary_logloss: 0.296543
[210]	valid_0's binary_logloss: 0.296476
[211]	valid_0's binary_logloss: 0.296352
[212]	valid_0's binary_logloss: 0.296308
[213]	valid_0's binary_logloss: 0.296261
[214]	valid_0's binary_logloss: 0.296264
[215]	valid_0's binary_logloss: 0.296222
[216]	valid_0's binary_logloss: 0.296236
[217]	valid_0's binary_logloss: 0.29617
[218]	valid_0's binary_logloss: 0.296138
[219]	valid_0's binary_logloss: 0.296101
[220]	valid_0's binary_logloss: 0.29604
[221]	valid_0's binary_logloss: 0.295985
[222]	valid_0's binary_logloss: 0.295813
[223]	valid_0's binary_logloss: 0.295755
[224]	valid_0's binary_logloss: 0.295713
[225]	valid_0's binary_logloss: 0.29558
[226]	valid_0's binary_logloss: 0.295318
[227]	valid_0's bina

In [ ]:
from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred) 
    f1 = f1_score(y_test, pred) 
    # ROC AUC 
    roc_auc = roc_auc_score(y_test, pred_proba) 
    
    print('Confusion Matrix') 
    print(confusion) 
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format( accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_dev, lgbm_preds, lgbm_pred_proba)

Confusion Matrix
[[435859  53529]
 [ 46504 255969]]
accuracy: 0.8737, precision: 0.8270, recall: 0.8463, f1: 0.8365, roc_auc: 0.9473


In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  lgbm_wrapper.predict_proba(X_test)[:,1]}).to_csv(os.path.abspath("../submission")+ '/LGBM(피쳐변경_1월)_0.9473.csv', index=False)

In [ ]:
joblib.dump(lgbm_wrapper, os.path.abspath("../models") + "/LGBM(변경후_1월).pkl")

['C:\\Users\\dmsdn\\학교수업\\머신러닝\\컴피티션\\ML 컴피티션_김은욱서하은\\models/LGBM(변경후_1월).pkl']

## (2월) 모델링

In [ ]:
# 쓰는 데이터셋만 빼고 주석 처리

#X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(01월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(01월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(01월).pkl")

X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(02월).pkl")
y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(02월).pkl")
X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(02월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(03월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(03월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(03월).pkl")

### BaggingClassifier(CatBoostClassifier)

In [ ]:
########BaggingClassifier(CatBoostClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

cat = BaggingClassifier(base_estimator = CatBoostClassifier(), random_state=42)

cat_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    cat.fit(X_tr, y_tr)
    
    pred = cat.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(cat.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    cat_pred += sub_pred

In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  cat_pred}).to_csv(os.path.abspath("../submission")+ '/CAT(피쳐변경_2월)_0.9558.csv', index=False)

### BaggingClassifier(RandomForestClassifier)

In [ ]:
########BaggingClassifier(RandomForestClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

rf = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state=42)

rf_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    rf.fit(X_tr, y_tr)
    
    pred = rf.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(rf.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    rf_pred += sub_pred

BaggingClassifier 평균 AUC : 0.9314623473590823


In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  rf_pred}).to_csv(os.path.abspath("../submission")+ '/RF(피쳐변경_2월)_0.9441.csv', index=False)

### Early_stopping(LGBMClassifier)

In [ ]:
########Early_stopping(LGBMClassifier)2등 코드########

X_tr, X_dev, y_tr, y_dev = train_test_split(X_train, y_train, test_size=0.4, random_state=77) # test size 올리기위해 0.4

evals = [(X_dev, y_dev)] 

lgbm_wrapper = LGBMClassifier(n_estimators = 400,random_state=42) # 400번 학습

lgbm_wrapper.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_dev)
lgbm_pred_proba = lgbm_wrapper.predict_proba(X_dev)[:,1]

[1]	valid_0's binary_logloss: 0.601747
[2]	valid_0's binary_logloss: 0.561826
[3]	valid_0's binary_logloss: 0.528969
[4]	valid_0's binary_logloss: 0.501532
[5]	valid_0's binary_logloss: 0.478444
[6]	valid_0's binary_logloss: 0.458919
[7]	valid_0's binary_logloss: 0.441959
[8]	valid_0's binary_logloss: 0.427246
[9]	valid_0's binary_logloss: 0.414647
[10]	valid_0's binary_logloss: 0.403687
[11]	valid_0's binary_logloss: 0.394045
[12]	valid_0's binary_logloss: 0.385531
[13]	valid_0's binary_logloss: 0.37822
[14]	valid_0's binary_logloss: 0.371778
[15]	valid_0's binary_logloss: 0.365931
[16]	valid_0's binary_logloss: 0.360736
[17]	valid_0's binary_logloss: 0.356216
[18]	valid_0's binary_logloss: 0.352197
[19]	valid_0's binary_logloss: 0.348355
[20]	valid_0's binary_logloss: 0.344945
[21]	valid_0's binary_logloss: 0.341945
[22]	valid_0's binary_logloss: 0.339128
[23]	valid_0's binary_logloss: 0.336304
[24]	valid_0's binary_logloss: 0.333973
[25]	valid_0's binary_logloss: 0.33161
[26]	valid_

[206]	valid_0's binary_logloss: 0.278052
[207]	valid_0's binary_logloss: 0.277933
[208]	valid_0's binary_logloss: 0.277839
[209]	valid_0's binary_logloss: 0.277726
[210]	valid_0's binary_logloss: 0.277678
[211]	valid_0's binary_logloss: 0.277604
[212]	valid_0's binary_logloss: 0.277581
[213]	valid_0's binary_logloss: 0.277554
[214]	valid_0's binary_logloss: 0.2775
[215]	valid_0's binary_logloss: 0.277425
[216]	valid_0's binary_logloss: 0.277279
[217]	valid_0's binary_logloss: 0.277202
[218]	valid_0's binary_logloss: 0.277128
[219]	valid_0's binary_logloss: 0.277107
[220]	valid_0's binary_logloss: 0.277074
[221]	valid_0's binary_logloss: 0.277008
[222]	valid_0's binary_logloss: 0.27688
[223]	valid_0's binary_logloss: 0.276756
[224]	valid_0's binary_logloss: 0.276641
[225]	valid_0's binary_logloss: 0.276608
[226]	valid_0's binary_logloss: 0.276531
[227]	valid_0's binary_logloss: 0.276421
[228]	valid_0's binary_logloss: 0.276376
[229]	valid_0's binary_logloss: 0.276313
[230]	valid_0's bin

In [ ]:
from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred) 
    f1 = f1_score(y_test, pred) 
    # ROC AUC 
    roc_auc = roc_auc_score(y_test, pred_proba) 
    
    print('Confusion Matrix') 
    print(confusion) 
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format( accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_dev, lgbm_preds, lgbm_pred_proba)

Confusion Matrix
[[348816  39055]
 [ 32971 181580]]
accuracy: 0.8804, precision: 0.8230, recall: 0.8463, f1: 0.8345, roc_auc: 0.9518


In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  lgbm_wrapper.predict_proba(X_test)[:,1]}).to_csv(os.path.abspath("../submission")+ '/LGBM(피쳐변경_2월)_0.9518.csv', index=False)

In [ ]:
joblib.dump(lgbm_wrapper, os.path.abspath("../models") + "/LGBM(변경후_2월).pkl")

['C:\\Users\\dmsdn\\학교수업\\머신러닝\\컴피티션\\ML 컴피티션_김은욱서하은\\models/LGBM(변경후_2월).pkl']

## (3월) 모델링

In [ ]:
# 쓰는 데이터셋만 빼고 주석 처리

#X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(01월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(01월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(01월).pkl")

#X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(02월).pkl")
#y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(02월).pkl")
#X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(02월).pkl")

X_train = joblib.load(os.path.abspath("../input") + "/X_train_변경(03월).pkl")
y_train = joblib.load(os.path.abspath("../input") + "/y_train_변경(03월).pkl")
X_test = joblib.load(os.path.abspath("../input") + "/X_test_변경(03월).pkl")

### BaggingClassifier(CatBoostClassifier)

In [ ]:
########BaggingClassifier(CatBoostClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

cat = BaggingClassifier(base_estimator = CatBoostClassifier(), random_state=42)

cat_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    cat.fit(X_tr, y_tr)
    
    pred = cat.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(cat.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    cat_pred += sub_pred

In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  cat_pred}).to_csv(os.path.abspath("../submission")+ '/CAT(피쳐변경_3월)_0.9587.csv', index=False)

### BaggingClassifier(RandomForestClassifier)

In [ ]:
########BaggingClassifier(RandomForestClassifier)데이콘코드########
# https://dacon.io/competitions/open/235538/codeshare/2725?page=1&dtype=recent

kf = KFold(n_splits = 5, shuffle = True, random_state = 77)

rf = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state=42)

rf_pred = np.zeros((X_test.shape[0]))

auc_list = []

for train_index, dev_index in kf.split(X_train, y_train):
    X_tr, y_tr = X_train.iloc[train_index], y_train.iloc[train_index]
    X_dev, y_dev = X_train.iloc[dev_index], y_train.iloc[dev_index]

    rf.fit(X_tr, y_tr)
    
    pred = rf.predict_proba(X_dev)[:, 1]

    auc = roc_auc_score(y_dev, pred)
    
    auc_list.append(auc)
    
    sub_pred = np.array(rf.predict_proba(X_test)[:, 1]) / 5 #fold 수
    
    rf_pred += sub_pred

BaggingClassifier 평균 AUC : 0.9314623473590823


In [ ]:
print(f'AUC_5fold_mean : {np.mean(auc_list)}')

In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  rf_pred}).to_csv(os.path.abspath("../submission")+ '/RF(피쳐변경_3월)_0.9482.csv', index=False)

## Early_stopping(LGBMClassifier)

In [ ]:
########Early_stopping(LGBMClassifier)2등 코드########

X_tr, X_dev, y_tr, y_dev = train_test_split(X_train, y_train, test_size=0.4, random_state=77) # test size 올리기위해 0.4

evals = [(X_dev, y_dev)] 

lgbm_wrapper = LGBMClassifier(n_estimators = 400,random_state=42) # 400번 학습

lgbm_wrapper.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
lgbm_preds = lgbm_wrapper.predict(X_dev)
lgbm_pred_proba = lgbm_wrapper.predict_proba(X_dev)[:,1]

[1]	valid_0's binary_logloss: 0.600224
[2]	valid_0's binary_logloss: 0.55902
[3]	valid_0's binary_logloss: 0.525067
[4]	valid_0's binary_logloss: 0.496808
[5]	valid_0's binary_logloss: 0.472853
[6]	valid_0's binary_logloss: 0.452371
[7]	valid_0's binary_logloss: 0.434968
[8]	valid_0's binary_logloss: 0.419541
[9]	valid_0's binary_logloss: 0.406639
[10]	valid_0's binary_logloss: 0.395301
[11]	valid_0's binary_logloss: 0.385216
[12]	valid_0's binary_logloss: 0.376593
[13]	valid_0's binary_logloss: 0.368888
[14]	valid_0's binary_logloss: 0.362198
[15]	valid_0's binary_logloss: 0.356219
[16]	valid_0's binary_logloss: 0.350752
[17]	valid_0's binary_logloss: 0.345806
[18]	valid_0's binary_logloss: 0.341442
[19]	valid_0's binary_logloss: 0.337639
[20]	valid_0's binary_logloss: 0.334037
[21]	valid_0's binary_logloss: 0.330689
[22]	valid_0's binary_logloss: 0.327815
[23]	valid_0's binary_logloss: 0.32532
[24]	valid_0's binary_logloss: 0.322902
[25]	valid_0's binary_logloss: 0.320829
[26]	valid_

[204]	valid_0's binary_logloss: 0.267462
[205]	valid_0's binary_logloss: 0.267386
[206]	valid_0's binary_logloss: 0.267345
[207]	valid_0's binary_logloss: 0.267884
[208]	valid_0's binary_logloss: 0.267359
[209]	valid_0's binary_logloss: 0.267318
[210]	valid_0's binary_logloss: 0.267247
[211]	valid_0's binary_logloss: 0.26719
[212]	valid_0's binary_logloss: 0.267155
[213]	valid_0's binary_logloss: 0.267139
[214]	valid_0's binary_logloss: 0.267049
[215]	valid_0's binary_logloss: 0.267009
[216]	valid_0's binary_logloss: 0.266894
[217]	valid_0's binary_logloss: 0.266798
[218]	valid_0's binary_logloss: 0.266763
[219]	valid_0's binary_logloss: 0.266732
[220]	valid_0's binary_logloss: 0.266684
[221]	valid_0's binary_logloss: 0.266624
[222]	valid_0's binary_logloss: 0.266511
[223]	valid_0's binary_logloss: 0.26647
[224]	valid_0's binary_logloss: 0.266413
[225]	valid_0's binary_logloss: 0.266388
[226]	valid_0's binary_logloss: 0.266329
[227]	valid_0's binary_logloss: 0.266294
[228]	valid_0's bi

In [ ]:
from sklearn.metrics import *

def get_clf_eval(y_test, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred) 
    f1 = f1_score(y_test, pred) 
    # ROC AUC 
    roc_auc = roc_auc_score(y_test, pred_proba) 
    
    print('Confusion Matrix') 
    print(confusion) 
    print('accuracy: {0:.4f}, precision: {1:.4f}, recall: {2:.4f}, f1: {3:.4f}, roc_auc: {4:.4f}'.format( accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_dev, lgbm_preds, lgbm_pred_proba)

Confusion Matrix
[[272016  29360]
 [ 24601 142241]]
accuracy: 0.8848, precision: 0.8289, recall: 0.8525, f1: 0.8406, roc_auc: 0.9550


In [ ]:
pd.DataFrame({'ID': test_id["ID"], 'STATUS':  lgbm_wrapper.predict_proba(X_test)[:,1]}).to_csv(os.path.abspath("../submission")+ '/LGBM(피쳐변경_3월)_0.9550.csv', index=False)

In [ ]:
joblib.dump(lgbm_wrapper, os.path.abspath("../models") + "/LGBM(변경후_3월).pkl")

['C:\\Users\\dmsdn\\학교수업\\머신러닝\\컴피티션\\ML 컴피티션_김은욱서하은\\models/LGBM(변경후_3월).pkl']

# 3. 앙상블

## 3-1. 피쳐 변경 전 모델별 1,2,3월 산술 평균

In [ ]:
############################### RF ###############################
from functools import reduce

a = pd.read_csv(os.path.abspath("../submission") + "/RF(변경전_1월)_0.9314.csv")
b = pd.read_csv(os.path.abspath("../submission") + "/RF(변경전_2월)_0.9243.csv")
c = pd.read_csv(os.path.abspath("../submission") + "/RF(변경전_3월)_0.9264.csv")

dfs = [a,b,c]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
means = df_merge.iloc[:, 1:].mean(axis=1)

final = pd.concat([df_merge.iloc[:, :1], means], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)

final.to_csv(os.path.abspath("../submission") + "/RF(변경전_1,2,3월).csv", index = False)

In [ ]:
############################### CAT ###############################
from functools import reduce

a = pd.read_csv(os.path.abspath("../submission") + "/CAT(변경전_1월)_0.9294.csv")
b = pd.read_csv(os.path.abspath("../submission") + "/CAT(변경전_2월)_0.9285.csv")
c = pd.read_csv(os.path.abspath("../submission") + "/CAT(변경전_3월)_0.9308.csv")

dfs = [a,b,c]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
means = df_merge.iloc[:, 1:].mean(axis=1)

final = pd.concat([df_merge.iloc[:, :1], means], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)

final.to_csv(os.path.abspath("../submission") + "/CAT(변경전_1,2,3월).csv", index = False)

In [ ]:
############################### LGBM ###############################
from functools import reduce

a = pd.read_csv(os.path.abspath("../submission") + "/LGBM(변경전_1월)_0.9246.csv")
b = pd.read_csv(os.path.abspath("../submission") + "/LGBM(변경전_2월)_0.9259.csv")
c = pd.read_csv(os.path.abspath("../submission") + "/LGBM(변경전_3월)_0.9286.csv")

dfs = [a,b,c]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
means = df_merge.iloc[:, 1:].mean(axis=1)

final = pd.concat([df_merge.iloc[:, :1], means], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)

final.to_csv(os.path.abspath("../submission") + "/LGBM(변경전_1,2,3월).csv", index = False)

## 3-2. 피쳐 변경 후 모델별 1,2,3월 산술 평균

In [ ]:
############################### RF ###############################
from functools import reduce

a = pd.read_csv(os.path.abspath("../submission") + "/RF(피쳐변경_1월)_0.9449.csv")
b = pd.read_csv(os.path.abspath("../submission") + "/RF(피쳐변경_2월)_0.9441.csv")
c = pd.read_csv(os.path.abspath("../submission") + "/RF(피쳐변경_3월)_0.9482.csv")

dfs = [a,b,c]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
means = df_merge.iloc[:, 1:].mean(axis=1)

final = pd.concat([df_merge.iloc[:, :1], means], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)

final.to_csv(os.path.abspath("../submission") + "/RF(피쳐변경_1,2,3월).csv", index = False)

In [ ]:
############################### CAT ###############################
from functools import reduce

a = pd.read_csv(os.path.abspath("../submission") + "/CAT(피쳐변경_1월)_0.9535.csv")
b = pd.read_csv(os.path.abspath("../submission") + "/CAT(피쳐변경_2월)_0.9558.csv")
c = pd.read_csv(os.path.abspath("../submission") + "/CAT(피쳐변경_3월)_0.9587.csv")

dfs = [a,b,c]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
means = df_merge.iloc[:, 1:].mean(axis=1)

final = pd.concat([df_merge.iloc[:, :1], means], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)

final.to_csv(os.path.abspath("../submission") + "/CAT(피쳐변경_1,2,3월).csv", index = False)

In [ ]:
############################### LGBM ###############################
from functools import reduce

a = pd.read_csv(os.path.abspath("../submission") + "/LGBM(피쳐변경_1월)_0.9473.csv")
b = pd.read_csv(os.path.abspath("../submission") + "/LGBM(피쳐변경_2월)_0.9518.csv")
c = pd.read_csv(os.path.abspath("../submission") + "/LGBM(피쳐변경_3월)_0.9550.csv")

dfs = [a,b,c]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)
means = df_merge.iloc[:, 1:].mean(axis=1)

final = pd.concat([df_merge.iloc[:, :1], means], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)

final.to_csv(os.path.abspath("../submission") + "/LGBM(피쳐변경_1,2,3월).csv", index = False)

## 3-3 (모델별 월 평균 submission) 6개 가중 평균

In [ ]:
## 피쳐 변경후 평균한 서브미션의 LB 점수가 높기때문에 가중을 더둔다
## 모델 중에선 RF->CAT->LGBM 순으로 높기 때문에 가중을 더 둔다

In [ ]:
a = pd.read_csv(os.path.abspath("../submission") + "/RF(변경전_1,2,3월).csv")
b = pd.read_csv(os.path.abspath("../submission") + "/CAT(변경전_1,2,3월).csv")
c = pd.read_csv(os.path.abspath("../submission") + "/LGBM(변경전_1,2,3월).csv")
d = pd.read_csv(os.path.abspath("../submission") + "/RF(피쳐변경_1,2,3월).csv")
e = pd.read_csv(os.path.abspath("../submission") + "/CAT(피쳐변경_1,2,3월).csv")
f = pd.read_csv(os.path.abspath("../submission") + "/LGBM(피쳐변경_1,2,3월).csv")

In [ ]:
dfs = [a,b,c,d,e,f]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='ID'), dfs)

In [ ]:
# 가중치 정하기
## 피쳐 변경후 평균한 서브미션의 LB 점수가 높기때문에 가중을 더둔다
## 모델 중에선 RF->CAT->LGBM 순으로 높기 때문에 가중을 더 둔다
## 순서대로 0.1/0.1/0.1/0.15/0.15/0.4
df_merge["가중평균"] = a["STATUS"]*0.1+ b["STATUS"]*0.1 + c["STATUS"]*0.1 + d["STATUS"]*0.4 + e["STATUS"]*0.15 + f["STATUS"]*0.15

In [ ]:
final = pd.concat([df_merge.iloc[:, :1], df_merge.iloc[:, -1:]], axis=1)
final.rename(columns = {final.columns[1]:'STATUS'}, inplace = True)
final.to_csv(os.path.abspath("../submission") + "/final_score_pmean.csv", index = False)

In [ ]:
############## 최종 LB_score = 0.90603 ##############